<a href="https://colab.research.google.com/github/Indian1999/csutortok_17/blob/main/LogiGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://lsc.io/lesson11ai2

---



In [ ]:
import json
import string
import random
import nltk
import numpy as np
from nltk import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Dense, Dropout
nltk.download("punkt") #Írásjelek
nltk.download("punkt_tab")
nltk.download("wordnet") #Lemmatizációhoz

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
data_file = open("intents.json").read()
data = json.loads(data_file)
print(data)

{'intents': [{'tag': 'hello', 'patterns': ['Hello', 'Hi there', 'Good morning', "What's up"], 'responses': ['Hey!', 'Hello', 'Hi!', 'Good morning!'], 'context': ''}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'job', 'patterns': ['What is your job', 'What is your work'], 'responses': ['My job is to make you feel like everything is okay.', 'I work to serve you as well as possible'], 'context': ''}, {'tag': 'age', 'patterns': ['What is your age', 'How old are you', 'When were you born'], 'responses': ['I was born in 2021'], 'context': ''}, {'tag': 'feeling', 'patterns': ['How are you today', 'How are you'], 'responses': ['I am feeling good, you?', 'Very good and you?', "Actually, I'm okay and you?"], 'context': ''}, {'tag': 'good', 'patterns': ['I am good too', 'I feel fine', 'Good !', 'Fine', 'I am good', 'I am great', 'great'], 'responses': ['That is perfect!', "So, ever

In [ ]:
words = []
classes = []
data_x = []
data_y = []
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        data_x.append(pattern)
        data_y.append(intent["tag"])
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

In [ ]:
print(words)
print(classes)
print(data_x)
print(data_y)

['Hello', 'Hi', 'there', 'Good', 'morning', 'What', "'s", 'up', 'What', 'is', 'your', 'job', 'What', 'is', 'your', 'work', 'What', 'is', 'your', 'age', 'How', 'old', 'are', 'you', 'When', 'were', 'you', 'born', 'How', 'are', 'you', 'today', 'How', 'are', 'you', 'I', 'am', 'good', 'too', 'I', 'feel', 'fine', 'Good', '!', 'Fine', 'I', 'am', 'good', 'I', 'am', 'great', 'great', 'I', 'am', 'feeling', 'bad', 'No', 'I', 'am', 'sad', 'No', 'What', 'can', 'you', 'do', 'What', 'can', 'I', 'ask', 'you', 'Can', 'you', 'help', 'me', 'Are', 'you', 'a', 'girl', 'You', 'are', 'a', 'women', 'Are', 'you', 'a', 'men', 'Are', 'you', 'a', 'boy', 'Thank', 'you', 'Thank', 'you', 'very', 'much', 'thanks', 'Goodbye', 'Good', 'afternoon', 'Bye', 'Where', 'do', 'you', 'live', 'What', 'are', 'you', 'doing', 'Can', 'you', 'wait', '2', 'minutes', 'Please', 'wait', 'Wait', '2', 'secs', 'please', 'Are', 'you', 'still', 'there', '?', 'Are', 'you', 'here', '?']
['hello', 'noanswer', 'job', 'age', 'feeling', 'good', 'b

In [ ]:
lemmatizer = WordNetLemmatizer() # szótövesítés
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
print(words)
print(classes)

["'s", '2', 'a', 'afternoon', 'age', 'am', 'are', 'ask', 'bad', 'born', 'boy', 'bye', 'can', 'do', 'doing', 'feel', 'feeling', 'fine', 'girl', 'good', 'goodbye', 'great', 'hello', 'help', 'here', 'hi', 'how', 'i', 'is', 'job', 'live', 'me', 'men', 'minute', 'morning', 'much', 'no', 'old', 'please', 'sad', 'sec', 'still', 'thank', 'thanks', 'there', 'today', 'too', 'up', 'very', 'wait', 'were', 'what', 'when', 'where', 'woman', 'work', 'you', 'your']
['action', 'actions', 'age', 'bad', 'city', 'feeling', 'good', 'goodbye', 'hello', 'job', 'men', 'noanswer', 'still there', 'thanks', 'wait', 'women']


In [ ]:
training = []
out_empty = [0 for i in range(len(classes))]

for i in range(len(data_x)):
    bow = [] # bag of words
    text = lemmatizer.lemmatize(data_x[i].lower())
    for word in words:
        if word in text:
            bow.append(1)
        else:
            bow.append(0)
    output_row = list(out_empty)
    output_row[classes.index(data_y[i])] = 1
    training.append([bow, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))
print(train_x)
print(train_x.shape)

[[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 1 1]
 ...
 [0 0 1 ... 0 1 0]
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 1 0]]
(41, 58)


In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(train_x.shape[1],) , activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         7,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │         1,040 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,848 (65.81 KB)

 Trainable params: 16,848 (65.81 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_x, train_y, epochs=200, batch_size=8)

Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9013 - loss: 0.4223
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8161 - loss: 0.4099
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8714 - loss: 0.5151
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7759 - loss: 0.5987
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9639 - loss: 0.3097 
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9074 - loss: 0.4008 
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9282 - loss: 0.3338 
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9208 - loss: 0.3499  
Epoch 9/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9342 - loss: 0.4256  
Epoch 10/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9401 - loss: 0.3033 
Epoch 11/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7775 - loss: 0.5135  
Epoch 12/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9790 - 